### This notebook will only work with th data which already clean by the EDA notebook.

In [1]:
#base import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Model Import
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn import svm
#preprocessing import
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
#accuracy calculate import
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
ebay_clean= pd.read_csv('../data/cleaned/Ebay_cleaned.csv', index_col=0)
pd.set_option('display.max_columns', None)
ebay_clean.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,package_size,record_number,distance,handling_date,shipping_date,total_time,pay_year,pay_month,pay_date
0,1,25454,3,2019-03-27,0,0,3,5,97219,49040,13,28,1,2019-03-24,2019-03-29,5,1,1,3002,3,2,5,2019,3,24
1,0,6727381,2,2018-06-03,0,3,3,5,11415-3528,62521,0,20,1,2018-06-02,2018-06-05,0,4,2,1283,1,2,3,2018,6,2
2,1,18507,1,2019-01-08,0,4,3,5,27292,53010,1,20,1,2019-01-06,2019-01-10,9,4,3,1104,2,2,4,2019,1,6
3,1,4677,1,2018-12-18,0,0,3,5,90703,80022,1,36,1,2018-12-17,2018-12-21,8,4,4,1353,1,3,4,2018,12,17
4,1,4677,1,2018-07-28,0,0,3,5,90703,55070,1,25,1,2018-07-27,2018-07-30,3,4,5,2456,1,2,3,2018,7,27


In [3]:
ebay_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296420 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   b2c_c2c                    296420 non-null  int64 
 1   seller_id                  296420 non-null  int64 
 2   declared_handling_days     296420 non-null  int64 
 3   acceptance_scan_timestamp  296420 non-null  object
 4   shipment_method_id         296420 non-null  int64 
 5   shipping_fee               296420 non-null  int64 
 6   carrier_min_estimate       296420 non-null  int64 
 7   carrier_max_estimate       296420 non-null  int64 
 8   item_zip                   296420 non-null  object
 9   buyer_zip                  296420 non-null  object
 10  category_id                296420 non-null  int64 
 11  item_price                 296420 non-null  int64 
 12  quantity                   296420 non-null  int64 
 13  payment_datetime           296420 non-null  

In [4]:
X= ebay_clean.drop(columns=['total_time','payment_datetime', 'delivery_date', 'acceptance_scan_timestamp', 'item_zip', 'buyer_zip', 'shipping_date', 'handling_date'])
y= ebay_clean['total_time']

In [5]:
print(y.shape)
print(X.shape)

(296420,)
(296420, 17)


In [6]:
#split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Scale data


In [7]:
scaler= StandardScaler()
X_train_ss= scaler.fit_transform(X_train, y_train)
X_test_ss= scaler.transform(X_test)

In [8]:
y_true= y_test.values

In [9]:
def define_late(pred, y_actual= y_true):
    ontime= 0
    late= 0
    early= 0
    for i in range(len(pred)):
        if pred[i] == y_true[i]:
            ontime+=1
        elif pred[i]> y_true[i]:
            early+=1
        else:
            late+=1
        
    print(f'Ontime= {ontime}; {ontime/len(y_true)*100}')
    print(f'Late= {late}; {late/len(y_true)*100}')
    print(f'Early= {early}; {early/len(y_true)*100}')

Note:
pred > true-> early: 
pred = true-> ontime: 
pred < true-> late: 

#### LinearRegression model

In [10]:
#Initialize
linear_model= LinearRegression()
#fit model
linear_model.fit(X_train_ss, y_train)
linear_preds= linear_model.predict(X_test_ss)
linear_train_pred= linear_model.predict(X_train_ss)
linear_train_pred= np.round(linear_train_pred)
linear_preds= np.round(linear_preds)
linear_score_test=  linear_model.score(X_test_ss, y_test)
linear_score_train=  linear_model.score(X_train_ss, y_train)
linear_accuracy_test= accuracy_score(y_test, linear_preds)
linear_accuracy_train= accuracy_score(y_train, linear_train_pred)

In [11]:
print(f'Accuracy score test:  {linear_accuracy_test}')
print(f'Accuracy score train: {linear_accuracy_train}')

Accuracy score test:  0.2226907766007692
Accuracy score train: 0.22248414411982997


In [12]:
define_late(linear_preds)

Ontime= 13202; 22.26907766007692
Late= 17617; 29.716280952702245
Early= 28465; 48.014641387220834


3 window:

1-3 days: 1
4-7 days: 2
>7 days: 3

#### Ridge model

In [13]:
# Initialize
ridge_model= Ridge(solver='lsqr')

# fit
ridge_model.fit(X_train_ss, y_train)
ridge_preds= ridge_model.predict(X_test_ss)
ridge_preds= np.round(ridge_preds)
ridge_train_pred= ridge_model.predict(X_train_ss)
ridge_train_pred= np.round(ridge_train_pred)
ridge_score_test= ridge_model.score(X_test_ss, y_test)
ridge_score_train= ridge_model.score(X_train_ss, y_train)
ridge_accuracy_test= accuracy_score(y_test, ridge_preds)
ridge_accuracy_train= accuracy_score(y_train, ridge_train_pred)


In [14]:
print(f'Ridge Accuracy score test:  {ridge_accuracy_test}')
print(f'Ridge Accuracy score train: {ridge_accuracy_train}')

Ridge Accuracy score test:  0.22265704068551379
Ridge Accuracy score train: 0.22248414411982997


In [15]:
define_late(ridge_preds)

Ontime= 13200; 22.26570406855138
Late= 17618; 29.717967748465018
Early= 28466; 48.0163281829836


In [16]:
Ontime= 13202; 22.26907766007692
Late= 17617; 29.716280952702245
Early= 28465; 48.014641387220834

48.014641387220834

#### XGboost

In [17]:
# initialize 
xg_boost= XGBRegressor()
#fit
xg_boost.fit(X_train_ss, y_train)
xg_pred= xg_boost.predict(X_test_ss)
xg_pred= np.round(xg_pred)
xg_train_pred= xg_boost.predict(X_train_ss)
xg_train_pred= np.round(xg_train_pred)
xg_train_accuracy=accuracy_score(y_train, xg_train_pred)
xg_test_accuracy= accuracy_score(y_test, xg_pred)
print(f'Train Score: {xg_boost.score(X_train_ss, y_train)}')
print(f'test score: {xg_boost.score(X_test_ss, y_test)}')
print(f'Test Accuracy : {xg_test_accuracy}')
print(f'Train Accuracy : {xg_train_accuracy}')

Train Score: 0.3261180310801296
test score: 0.15258448775584488
Test Accuracy : 0.23950813035557655
Train Accuracy : 0.24550890628162741


In [18]:
define_late(xg_pred)

Ontime= 14199; 23.950813035557655
Late= 17166; 28.95553606369341
Early= 27919; 47.09365090074893


In [19]:
def evaluate_loss(preds, actual= y_true):
    early_loss, late_loss = 0,0 
    for i in range(len(preds)):
        if preds[i] < actual[i]:
            #early shipment
            early_loss += actual[i] - preds[i]
        elif preds[i] > actual[i]:
            #late shipment
            late_loss += preds[i] - actual[i]
    loss = (1/len(preds)) * (0.4 * (early_loss) + 0.6 * (late_loss))
    return loss

In [20]:
print(f'rigde lost= {evaluate_loss(ridge_preds)} -- linear loss= {evaluate_loss(linear_preds)} --xg lost= {evaluate_loss(xg_pred)}')

rigde lost= 0.7799642399298293 -- linear loss= 0.7799709871128804 --xg lost= 0.736532622630052


In [21]:
accuracy = [linear_accuracy_test, linear_accuracy_train, ridge_accuracy_test, ridge_accuracy_train, xg_test_accuracy, xg_train_accuracy]
columns=['linear test', 'linear train', 'ridge test', 'ridge train', 'xg test', 'xg train']

##### Pipeline - GridSearch


In [22]:
estimators= [
    ('normalise', StandardScaler()),
    ('model', LinearRegression())
] 
my_pipe= Pipeline(estimators)

grid= [
    # {
    #     'model': [svm.SVC()],
    #     'normalise':[StandardScaler()], 
    #     'model__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
    #     'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
    #     'model__kernel':['linear', 'poly', 'rbf', 'sigmoid']
    # },
    {
        'model': [Ridge()],
        'normalise':[StandardScaler()],
        'model__solver':['aoto', 'lbfgs', 'lsqr']
    }, 
    {
        'model':[XGBRegressor()],
        'normalise':[StandardScaler()],
        'model__max_depth': range (2, 10, 1),
        'model__n_estimators': range(60, 220, 40),
        'model__learning_rate': [0.1, 0.01, 0.05]
    }
]
gridCV= GridSearchCV(my_pipe, grid)
fit_grid= gridCV.fit(X_train, y_train)

KeyboardInterrupt: 